### Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from transformers import AutoImageProcessor, SwinForImageClassification
from sklearn.metrics import accuracy_score
import torch

c:\adam_kacmar\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
torch.cuda.is_available(), torch.cuda.get_device_name(0)

### Hyperparameters

In [ ]:
batch_size = 16
learning_rate = 1e-4
num_epochs = 75
save_interval = 5
num_classes = 4
import wandb

sweep_config = {
    'method': 'grid', 
    'metric': {'name': 'val_loss', 'goal': 'minimize'},
    'parameters': {
        'batch_size': {'values': [16, 32, 64]},
        'learning_rate': {'values': [1e-5, 1e-4, 5e-4]}, 
        'num_epochs': {'value': num_epochs},  
        'num_classes': {'value': num_classes}, 
    }
}

sweep_id = wandb.sweep(sweep_config, project="swinv2-bc-hp-tiny-patch4-window8")

### Data Preprocessing

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
])

### Load Datasets


In [ ]:
train_dataset = datasets.ImageFolder("dataset/train", transform=transform)

# Define the split sizes
train_size = int(0.8 * len(train_dataset))  # 80% for training
val_size = len(train_dataset) - train_size  # 20% for validation

# Split the training dataset into train and validation subsets
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
test_dataset = datasets.ImageFolder("dataset/test", transform=transform)

### Training Function


In [ ]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    return avg_loss

### Validation Function


In [ ]:
def validate(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    avg_loss = total_loss / len(val_loader)
    return avg_loss, accuracy

### Main Function


In [ ]:
import os

os.makedirs('hp-models-tiny', exist_ok=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
def train_with_sweep():
    # Initialize W&B
    wandb.init()

    # Get hyperparameters from W&B
    config = wandb.config

    # Set up model
    model = SwinForImageClassification.from_pretrained(
        "microsoft/swinv2-tiny-patch4-window8-256", 
        num_labels=config.num_classes, 
        ignore_mismatched_sizes=True)
    model.to(device)    

    # Set up optimizer and criterion
    optimizer = optim.AdamW(
        model.parameters(), 
        lr=config.learning_rate
        )
    criterion = nn.CrossEntropyLoss()

    # Dynamically set batch size in DataLoader
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)

    patience = 10  # Stop training if no improvement for 'patience' epochs
    min_delta = 1e-4  # Minimum change in val_loss to qualify as improvement
    best_val_loss = float('inf')
    patience_counter = 0

    # Training loop
    for epoch in range(config.num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer)
        print(f"Epoch {epoch+1}/{config.num_epochs}")
        print(f"Train Loss: {train_loss:.4f}")

        # Validation
        val_loss, val_accuracy = validate(model, val_loader, criterion)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        # Log metrics to W&B
        wandb.log({
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_accuracy': val_accuracy,
            'epoch': epoch + 1
        })

        # Check early stopping criteria
        if val_loss < best_val_loss - min_delta:
            best_val_loss = val_loss
            patience_counter = 0  # Reset patience counter
            # Save the model if it's the best so far
            torch.save(model.state_dict(), f'hp-models-tiny/model_lr{config.learning_rate}_bs{config.batch_size}.pth')
            print(f"New best model saved. Validation Loss: {val_loss:.4f}")
        else:
            patience_counter += 1
            print(f"No improvement in validation loss for {patience_counter} epoch(s).")

        # Early stopping condition
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

    print("Evaluating the best model on the test dataset...")
    model.load_state_dict(torch.load(f'hp-models-tiny/model_lr{config.learning_rate}_bs{config.batch_size}.pth'))
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_accuracy = accuracy_score(all_labels, all_preds)
    print(f"Final Test Accuracy: {test_accuracy:.4f}")

    # Log final test accuracy to W&B
    wandb.log({'final_test_accuracy': test_accuracy})

wandb.agent(sweep_id, function=train_with_sweep)